# Customer Churn Prediction

This notebook covers:
- Data Cleaning
- Exploratory Data Analysis (EDA)
- Preprocessing Steps
- Model Building (in later stages)

In [ ]:
# Step 1: Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')